# Jug pouring

In Die Hard 3, Bruce Willis has to figure out how to measure exactly 4 gallons of water given only a 3 gallon jug and a 5 gallon jug. Solve this using graph search.

Assume that the only possible operations are:

1. Filling up the 3 gallon jug (+3, 0)
2. Filling up the 5 gallon jug (0, +5)
3. Pouring the 3 gallon jug into the 5 gallon jug (-3, +3), handling overflow
4. Pouring the 5 gallon jug into the 3 gallon jug (+3, -5), handling overflow

(This feels a bit like linear algebra.)

Plan:

1. Instead of precomputing the entire graph (not sure how many nodes there are), have a function that generates valid transitions
   - Can fill jugs up completely: Given `(a, b)`, if `a < 3` or `b < 5`, we move to `(3, b)` or `(a, 5)`
   - Can transfer between jugs: Given `(a, b)`, if `a, b > 0`, we move to `(min(a + b, 3), max(0, b - a))` or `(max(0, a - b), min(a + b, 3))`
   - Allow emptying the jugs? (Is this useful?)
2. Given these rules, try to traverse the entire graph with DFS and find important states


In [ ]:
from typing import Tuple, List, Iterator, Callable
from collections import deque

State = Tuple[int, int, str]

In [ ]:
def transitions(state: State, empty: bool = False) -> List[State]:
    x, y, _ = state

    states = [
        (3, y, "fill left"),
        (x, 5, "fill right"),
    ]
    if empty:
        states.extend(
            [
                (0, y, "empty left"),
                (x, 0, "empty right"),
            ]
        )
    if x > 0:
        states.append(
            (max(0, x - (5 - y)), min(y + 3, 5), "pour into right"),
        )
    if y > 0:
        states.append(
            (min(x + y, 3), max(0, y - (3 - x)), "pour into left"),
        )

    return states

In [ ]:
def traverse(
    start: State, transitions: Callable[[State], List[State]]
) -> Iterator[State]:
    visited = set()
    stack = [start]
    while stack:
        state = stack.pop()
        visited.add(state[:2])
        yield state
        for nxt in transitions(state):
            if nxt[:2] not in visited:
                stack.append(nxt)

In [ ]:
start = (0, 0, "start")

In [ ]:
for state in traverse(start, transitions):
    print(state)

In [ ]:
sorted(set(state[:2] for state in traverse(start, transitions)))

Emptying jugs is useful! Without it, you can't get to the goal state.


In [ ]:
def transitions_with_emptying(state: State) -> List[State]:
    return transitions(state, empty=True)

In [ ]:
for state in traverse((0, 0, "start"), transitions_with_emptying):
    print(state)

In [ ]:
sorted(set(state[:2] for state in traverse(start, transitions_with_emptying)))

What's the shortest path to the goal state?


In [ ]:
def search(
    start: State,
    predicate: Callable[[State], bool],
    transitions: Callable[[State], List[State]],
) -> List[List[State]]:
    visited = set()
    queue = deque([(start, [start])])
    paths: List[List[State]] = []
    shortest_path = float("inf")

    while queue:
        state, path = queue.popleft()
        visited.add(state)
        if predicate(state):
            # if len(path) < shortest_path:
            #     paths.clear()
            #     shortest_path = len(path)
            if len(path) <= shortest_path:
                paths.append(path)

        for nxt in transitions(state):
            if nxt not in visited:
                queue.append((nxt, path + [nxt]))

    return paths

In [ ]:
# No path exists!
search(start, lambda state: state[1] == 4, transitions)

In [ ]:
# All paths that reach the goal state
for path in search(start, lambda state: state[1] == 4, transitions_with_emptying):
    print(len(path), path)

In [ ]:
# Shortest path
search(start, lambda state: state[1] == 4, transitions_with_emptying)[0]

# Leetcode

It turns out a version of this problem is on Leetcode: https://leetcode.com/problems/water-and-jug-problem/description/


# Number theory

In trying out different possible jug sizes and target volumes, there seems to be a pattern. For it to be possible to reach a target, the jugs must be coprime. Otherwise, you can only reach targets that are multiples of `gcd(x, y)`. For instance, given `x = 2, y = 4`, the only possible states are `0, 2, 4, 6`.

My hunch is that given coprime `x, y`, the possible states are all values between `0..x + y`.
